In [1]:
import cv2 as cv
import glob
import numpy as np

In [17]:
CHECKERBOARD = (9, 6)   # dimensions of the _interior_ corners
SQUARE_SIZE = 1.0

# Containers to hold the points
# Object points (3D), like (0,0,0), (1,0,0), (2,0,0), ..., (6,5,0)
# These are the corner points in the world coord sys
objp = np.zeros((CHECKERBOARD[0] * CHECKERBOARD[1], 3), dtype=np.float32)
objp[:, :2] = np.mgrid[0:CHECKERBOARD[0], 0:CHECKERBOARD[1]].T.reshape(-1,2)

# Arrays to store object points and image points from all images
objpoints = []  # 3D point in real world space
imgpoints = []  # 2D points in image plane

In [18]:
dirname = "chessboard"
images = sorted(glob.glob(f"./data/{dirname}/img_*.jpg"))

print(f"Found {len(images)} images.")

Found 30 images.


In [19]:
# Set up termination criteria for corner subpixel refinement
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)

for fname in images:
    img = cv.imread(fname)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

    # Find the chessboard's interior corners
    ret, corners = cv.findChessboardCorners(gray, patternSize=CHECKERBOARD, corners=None)

    if ret:
        # If corners were successfully found
        # Add object points
        objpoints.append(objp)  # same for every image, since we know the geometry

        # Refine the detected image corners, and add them
        corners2 = cv.cornerSubPix(
            image=gray,
            corners=corners,
            winSize=(11, 11),
            zeroZone=(-1, -1),
            criteria=criteria,
        )
        imgpoints.append(corners2)
    else:
        print(f"Failed to detect chessboard in image '{fname}'.")
        objpoints.append(None)
        imgpoints.append(None)

Failed to detect chessboard in image './data/chessboard\img_26.jpg'.


In [24]:
idx = 20

img = cv.imread(images[idx])
corners = imgpoints[idx]

cv.drawChessboardCorners(
    image=img,
    patternSize=CHECKERBOARD,
    corners=corners,
    patternWasFound=True,
)
cv.imshow('img', cv.cvtColor(img, cv.COLOR_BGR2GRAY))
cv.waitKey(3*1_000)
cv.destroyAllWindows()

Now we have, for every image in our dataset, a collection of object points in the world coord sys and the corresponding collection of detected points in the image. We can use this entire collection of correspondences to fit our camera model; this is calibration. 

In [26]:
objpoints = [objp for objp in objpoints if objp is not None]
imgpoints = [imgp for imgp in imgpoints if imgp is not None]

ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(
    objectPoints=objpoints,
    imagePoints=imgpoints,
    imageSize=gray.shape[::-1],
    cameraMatrix=None,  # We want to fit this!
    distCoeffs=None,    # We want to fit this!
)
# ret - return code
# mtx - camera intrinsics matrix
# dist - distortion coefficients
# rvecs - (extrinsic) rotation vectors
# tvecs - (extrinsic) translation vectors

In [65]:
def print_mtx(mtx: np.ndarray) -> None:
    if not mtx.shape == (3,3):
        raise ValueError(f"Camera intrinsics matrix should have shape (3,3); got {mtx.shape}")
    print(f"[[ {int(mtx[0][0]):^6d}  {int(mtx[0][1]):^6d}  {int(mtx[0][2]):^6d} ]")
    print(f" [ {int(mtx[1][0]):^6d}  {int(mtx[1][1]):^6d}  {int(mtx[1][2]):^6d} ]")
    print(f" [ {int(mtx[2][0]):^6d}  {int(mtx[2][1]):^6d}  {int(mtx[2][2]):^6d} ]]")

print_mtx(mtx)

print(dist)

[[  1024     0      638   ]
 [   0      1027    375   ]
 [   0       0       1    ]]
[[ 0.11813414 -0.63363523 -0.00117985 -0.00254147  0.77097999]]


How can I interpret these values?
- $f_x = 1024$, $f_y = 1027$ are the focal lengths in pixel units.
    - These roughly correspond to horizontal and vertical FOVs as $64.0$ and $38.6$ degrees, respectively, which are plausible.
- $c_x = 638$, $c_y = 375$ are the location of the principal point in pixels.
    - Note that the center of the image is at $\left(1280, 720\right)/2 = \left(640, 360\right)$, so this is fairly close.

### How can we assess the calibration?

One method is **re-projection error**, which we calculate by projecting the object points to the image points by iterating the projection model forward; then we compare the results with the actual image points. 

In [56]:
mean_error = 0.0

for i in range(len(objpoints)):
    # Use the determined parameters to re-project the object points
    imgpoints_proj, _ = cv.projectPoints(
        objectPoints=objpoints[i],
        rvec=rvecs[i],
        tvec=tvecs[i],
        cameraMatrix=mtx,
        distCoeffs=dist,
    )
    # Calculate the norm
    error = cv.norm(imgpoints[i], imgpoints_proj, cv.NORM_L2) / len(imgpoints_proj)
    mean_error += error

mean_error /= len(objpoints)

print(f"Mean reprojection error: {mean_error:0.2f}")

Mean reprojection error: 0.13


### Question: What about real distances?

I noticed that `cv.calibrateCamera` returned extrinsic parameters `(rvecs, tvecs)` for each pose that was used during calibration, but I didn't think I told it anything about my real-world dimensions. It turns out that I _did_,  when I defined the world coordinate system (the planar system on the board).

Thus, if I redefine this coordinate system and use the actual physical units, the quantities that are fit by the calibration should be in actual physical units. For example, `tvecs[0]` will be the actual vector from the camera's origin to the chessboard origin at the time of my taking `image[0]`. 

### What can we do with this?

In [63]:
parts = fname.split('.')
'.'.join(parts[:-1]) + '_calib.jpg'

'./data/chessboard\\img_29.jpg'

In [66]:
fname = images[21]
img = cv.imread(fname)
h, w = img.shape[:2]
newcameramtx, roi = cv.getOptimalNewCameraMatrix(mtx, dist, (w,h), 1, (w,h))

# undistort
dst = cv.undistort(img, mtx, dist, None, newcameramtx)

# crop image
x, y, w, h = roi
dst = dst[y:y+h, x:x+w]

parts = fname.split('.')
dstname = '.'.join(parts[:-1]) + '_calib.jpg'
cv.imwrite(dstname, dst)

True

### What can't we do?

In the background of my calibration images, I have a whiteboard on my wall. For a given image, the upper-left corner and the upper-right corner of the whiteboard appear in a pair of pixels in the image. These pixels "pull back" to rays in 3D space under the camera model.

Unfortunately, this is as far as we can go. I was wondering if I could estimate the length of the whiteboard with this information. But from this single camera's perspective, there is no way to get depth information: we can tell what ray each pixel's actual object lies on, but we don't know about depth.

However, if we knew the plane that the whiteboard lies on --- i.e., the wall --- relative to the camera's coordinate system, then we _could_ do this.

Recall the equation $$ \tilde{x} = K\left[R\middle|t\right]\tilde{X} $$
That maps a point in $\mathbb{P}^3$ to a point in $\mathbb{P}^2$ --- that is, a ray in $\mathbb{R}^3$. Then we "normalize" this ray to get the point in pixel coordinates.

We can go a little bit in the other direction. Given a pixel $x = \left(u,v\right)$ in our image, we can lift it to $\tilde{x} = \left(u,v,1\right) \in \mathbb{P}^2$; then apply the intrinsics inverse to get $$K^{-1}\tilde{x}$$ which is a point in the normalized image plane, in the camera's coordinate system. Equivalently, this is a direction vector $\vec{d}$ for the ray from the camera center into 3D. The ray itself can be written $$ r\left(\lambda\right) = \lambda \left(K^{-1}\tilde{x}\right), \; \lambda > 0. $$

At this point, without knowing the extrinsics $\left(R,t\right)$ for the actual object represented by the pixel $\left(u,v\right)$, we can go no further in inverting this camera model.

However, say we happen to know $\left(R,t\right)$, which, again, relate the camera's coordinate system to the coordinate system of the wall (the world). Then we know the rays in 3D space --- in the camera's coordinate system --- on which the object points lie. We can
1. use intrinsics to pull these rays into the wall's coordinate system; then
2. model the wall as a plane in this system, and find where the rays hit the plane.

In detail, recall that extrinsics map from the wall/world coordinate system to camera coordinates as $$ X_c = RX_w + t.$$ We can easily invert this to get $$ X_w = R^T\left(X_c - t\right). $$

Let a ray's direction vector in the camera coordinate system be $\vec{r}_c$, and then the ray is $\lambda \vec{r}_c$. Then this ray pulls back to $$ \vec{r}_w = R^T\left(\lambda \vec{r}_c - t\right) = -R^T t + \lambda R^T \vec{r}_c $$ (note: $0\vec{r}_c$ is the camera center, which pulls back to $\vec{r}_w = -R^T t$ as expected). Lastly, write the plane of the wall as $$ \hat{n}^T X_w + d = 0, $$ plug in the parametric form of the ray $$ \hat{n}^T \vec{r}_w\left(\lambda\right) + d = 0, $$ and solve for $\lambda = \lambda^*$ to find where the ray hits the wall (plane). Then $\vec{r}_w\left(\lambda^*\right)$ is the object point that we sought (in the wall's coordinate system).